In [236]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [237]:
data = pd.read_csv("initial_data.csv")

#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder']]

x_max = x.max()
x_min = x.min()
y_max = y.max()
y_min = y.min()

#print(str(y_min) + "\n:\n" + str(y_max))

y_range = y_max - y_min

print(y_range)

NOx                  685.846823
PM 1                   5.547569
CO2                  160.188892
PM 2                  15.561156
Pressure cylinder    115.101073
dtype: float64


In [238]:
# Datenaufteilung in Training und Test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)



# Umwandlung in PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [239]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(8, 16)
        #self.layer2 = nn.Linear(16, 8)
        #self.layer3 = nn.Linear(12, 10)
        self.output = nn.Linear(16, 5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        #x = self.relu(self.layer2(x))
        #x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = NeuralNet()

In [240]:
# Verlustfunktion und Optimierer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trainingsparameter
num_epochs = 300
batch_size = 3

# Daten in Batches aufteilen
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [241]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward-Pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward-Pass und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modell speichern (optional)
torch.save(model.state_dict(), 'model.pth')

Epoch [10/300], Loss: 0.1544
Epoch [20/300], Loss: 0.4962
Epoch [30/300], Loss: 0.2150
Epoch [40/300], Loss: 0.2123
Epoch [50/300], Loss: 0.0738
Epoch [60/300], Loss: 0.0439
Epoch [70/300], Loss: 0.1704
Epoch [80/300], Loss: 0.1086
Epoch [90/300], Loss: 0.0547
Epoch [100/300], Loss: 0.0738
Epoch [110/300], Loss: 0.0492
Epoch [120/300], Loss: 0.0633
Epoch [130/300], Loss: 0.7510
Epoch [140/300], Loss: 0.1396
Epoch [150/300], Loss: 0.1153
Epoch [160/300], Loss: 0.1087
Epoch [170/300], Loss: 0.0085
Epoch [180/300], Loss: 0.1225
Epoch [190/300], Loss: 0.1965
Epoch [200/300], Loss: 0.0312
Epoch [210/300], Loss: 0.0650
Epoch [220/300], Loss: 0.0729
Epoch [230/300], Loss: 0.1363
Epoch [240/300], Loss: 0.0602
Epoch [250/300], Loss: 0.0186
Epoch [260/300], Loss: 0.0745
Epoch [270/300], Loss: 0.0865
Epoch [280/300], Loss: 0.1886
Epoch [290/300], Loss: 0.0634
Epoch [300/300], Loss: 0.0570


In [242]:
model.eval()  # Setzt das Modell in den Evaluationsmodus
with torch.no_grad():  # Keine Gradientenberechnung
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)
    #print(y_test)
    
    print(f'Test Loss: {test_loss.item():.4f}')
    y_pred = scaler_y.inverse_transform(y_pred)
    y_test = scaler_y.inverse_transform(y_test)
    for pred, test in zip(y_pred, y_test):
        print(str(pred) + ":" + str(test))

Test Loss: 0.4354
[38.88923029  0.52117083 18.8422769   2.35521152 54.30891802]:[45.11063706  0.37043239 13.33908759  1.34438471 50.75279046]
[127.99717254   2.99342779  65.13084665   9.69170347  72.02820745]:[144.31417996   3.2442979   63.07336518   9.38207516  71.47676054]
[78.55949241  1.94996514 33.92522264  6.62800625 62.22061233]:[66.86942417  4.09588526 36.29146793 16.28325513 60.49792953]
[80.50035026  2.22188088 43.89848757  8.81374164 62.35524058]:[94.23299035  2.83292106 38.50904745 10.53630092 59.95670593]
[50.40468278  1.2468531  23.54815474  5.81114129 54.86618818]:[41.90699086  0.76024322 22.40993205  4.24715129 54.66871374]
[94.7411168   2.69419485 49.13469223 10.61561499 64.76352359]:[84.29989805  3.41757813 48.34347086 14.36933964 66.03415315]
[530.35526748   1.08458558  75.13174254   2.1682828  108.21763291]:[465.21158656   0.99647182  76.01429871   2.44630606 107.39016145]
[195.12089255   3.21622458 100.53495453   5.49137409 102.65177744]:[165.12383548   3.63098576 

In [243]:
# build matrix of elementwise mean squared error

from sklearn.metrics import mean_squared_error


matrix = []
for pred, test in zip(y_pred, y_test):
    data_set_mse = []
    for p, t in zip(pred, test):
        #print(str(pred) + ":" + str(test))
        mse = mean_squared_error([p], [t])
        data_set_mse.append(mse)
    #print(data_set_mse)

    percentage_to_range = data_set_mse / y_range
    matrix.append(percentage_to_range)
#print(matrix)


In [244]:
# check mse per dataset row

from statistics import mean 

mean_for_predicted_dataset = []

for predicted_dataset in matrix:
    mean_for_predicted_dataset.append(mean(predicted_dataset))

print("\n")
for entry in mean_for_predicted_dataset:
    print(entry)

print("Mean:")
print(mean(mean_for_predicted_dataset))



0.08502407021302955
0.08695451669311216
1.41617646269859
0.1528513855982927
0.06271746070604456
0.23534108975632845
1.2409420039928099
0.3546098705108222
0.10843040615314829
0.13778299559181134
Mean:
0.3880830261913989


In [245]:
# Ceck Accuracy among features of test data

#print(y_pred.shape)

y_pred_t = np.transpose(y_pred)
y_test_t = np.transpose(y_test)

# print(y_pred_t.shape)
# print(y_test_t.shape)
featurewise_mean = []
for feature_t, feature_p in zip(y_test_t, y_pred_t):
    # print(feature_t)
    # print(feature_p)
    # print(mean_squared_error(feature_t, feature_p))
    featurewise_mean.append(mean_squared_error(feature_t, feature_p))

print(featurewise_mean)

print(y_range)
featurewise_mean_percentage = featurewise_mean / y_range
print("mse per feature in percentage to value range")
print(featurewise_mean_percentage)

[617.5133914290598, 0.6815258139403898, 10.705812730275088, 11.884536564535889, 9.971682765266163]
NOx                  685.846823
PM 1                   5.547569
CO2                  160.188892
PM 2                  15.561156
Pressure cylinder    115.101073
dtype: float64
mse per feature in percentage to value range
NOx                  0.900366
PM 1                 0.122851
CO2                  0.066832
PM 2                 0.763731
Pressure cylinder    0.086634
dtype: float64
